In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [3]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [4]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [5]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [6]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


In [7]:
#  dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [8]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [9]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [10]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [13]:
len(X_train)

623

In [14]:
len(X_test)

268

In [15]:
from sklearn.naive_bayes import GaussianNB

In [16]:
model = GaussianNB()

In [17]:
model.fit(X_train, y_train)

GaussianNB()

In [18]:
model.score(X_test,y_test)

0.753731343283582

In [19]:
X_test[0:10]

,Pclass,Age,Fare,female
331,1,45.500000,28.5000,0
322,2,30.000000,12.3500,1
699,3,42.000000,7.6500,0
590,3,35.000000,7.1250,0
240,3,29.699118,14.4542,1
768,3,29.699118,24.1500,0
751,3,6.000000,12.4750,0
457,1,29.699118,51.8625,1
769,3,32.000000,8.3625,0
59,3,11.000000,46.9000,0


In [20]:
y_test[0:10]

/tmp/ipykernel_9888/2966258366.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_test[0:10]


331    0
322    1
699    0
590    0
240    0
768    0
751    1
457    1
769    0
59     0
Name: Survived, dtype: int64

In [21]:
model.predict(X_test[0:10])

array([0, 1, 0, 0, 1, 0, 0, 1, 0, 0])

In [22]:
model.predict_proba(X_test[:10])

array([[0.78051997, 0.21948003],
       [0.17426254, 0.82573746],
       [0.96610824, 0.03389176],
       [0.96546484, 0.03453516],
       [0.31097341, 0.68902659],
       [0.96306084, 0.03693916],
       [0.93445029, 0.06554971],
       [0.0333917 , 0.9666083 ],
       [0.96484149, 0.03515851],
       [0.91862937, 0.08137063]])

In [23]:
# Calculate the score using cross validation
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.8       , 0.816     , 0.768     , 0.74193548, 0.81451613])